# Install Packages and Setup Variables


In [1]:
!pip install -q llama-index==0.12.21 llama-index-vector-stores-chroma==0.4.1 llama-index-llms-gemini==0.4.1 google-generativeai==0.5.4 openai==1.59.8 chromadb==0.6.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.6/455.6 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 602.1/602.1 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 679.1/679.1 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 2.6 MB/s eta 0:

In [2]:
import os

# Set the following API Keys in the Python environment. Will be used later.
# os.environ["OPENAI_API_KEY"] = "[OPENAI_API_KEY]"
# os.environ["GOOGLE_API_KEY"] = "<YOUR_API_KEY>"

from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('openai_api_key')
os.environ["GOOGLE_API_KEY"] = userdata.get('Google_api_key')

## Language Model and Embedding Model

In [3]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.gemini import Gemini
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI

# Settings.llm = OpenAI(temperature=0, model="gpt-4o-mini")

Settings.llm = Gemini(model="models/gemini-1.5-flash", temperature=1, max_tokens=512)
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

**Note: You can create a vector store from scratch using the code below, or you can load it from Hugging Face using the code provided in this notebook.**

# Create a Vector Store


In [4]:
import chromadb

# create client and a new collection
# chromadb.EphemeralClient saves data in-memory.
chroma_client = chromadb.PersistentClient(path="./ai_tutor_knowledge")
chroma_collection = chroma_client.create_collection("ai_tutor_knowledge")

In [5]:
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core.storage.storage_context import StorageContext

# Define a storage context object using the created vector database.
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Load the Dataset (JSON)


## Download


In [6]:
from huggingface_hub import hf_hub_download
file_path = hf_hub_download(repo_id="jaiganesan/ai_tutor_knowledge", filename="ai_tutor_knowledge.jsonl",repo_type="dataset",local_dir="/content")

ai_tutor_knowledge.jsonl:   0%|          | 0.00/6.96M [00:00<?, ?B/s]

# Read File


In [7]:
import json
with open(file_path, "r") as file:
    ai_tutor_knowledge = [json.loads(line) for line in file]

len(ai_tutor_knowledge)

762

# Convert to Document obj


In [8]:
from typing import List
from llama_index.core import Document

def create_docs_from_list(data_list: List[dict]) -> List[Document]:
    documents = []
    for data in data_list:
        documents.append(
            Document(
                doc_id=data["doc_id"],
                text=data["content"],
                metadata={  # type: ignore
                    "url": data["url"],
                    "title": data["name"],
                    "tokens": data["tokens"],
                    "source": data["source"],
                },
                excluded_llm_metadata_keys=[
                    "title",
                    "tokens",
                    "source",
                ],
                excluded_embed_metadata_keys=[
                    "url",
                    "tokens",
                    "source",
                ],
            )
        )
    return documents

doc = create_docs_from_list(ai_tutor_knowledge)

# Transforming


In [9]:
from llama_index.core.node_parser import SentenceWindowNodeParser

# create the sentence window node parser
node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=3,
    include_metadata=True,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
)

In [10]:
documents = [i for i in doc if i.metadata['tokens']<8000]
nodes = node_parser.get_nodes_from_documents(documents)

In [11]:
len(nodes)

22083

In [12]:
nodes[0]

TextNode(id_='0aca9d92-f9b6-4595-8040-753fcd85a76d', embedding=None, metadata={'url': 'https://towardsai.net/p/machine-learning/bert-huggingface-model-deployment-using-kubernetes-github-repo-03-07-2024', 'title': 'BERT HuggingFace Model Deployment using Kubernetes [ Github Repo]  03/07/2024', 'tokens': 768, 'source': 'tai_blog', 'window': 'Github Repo : https://github.com/vaibhawkhemka/ML-Umbrella/tree/main/MLops/Model_Deployment/Bert_Kubernetes_deployment   Model development is useless if you dont deploy it to production  which comes with a lot of issues of scalability and portability.    I have deployed a basic BERT model from the huggingface transformer on Kubernetes with the help of docker  which will give a feel of how to deploy and manage pods on production.    Model Serving and Deployment:ML Pipeline:Workflow:   Model server (using FastAPI  uvicorn) for BERT uncased model    Containerize model and inference scripts to create a docker image    Kubernetes deployment for these mode

## Index creation

In [13]:
from llama_index.core import VectorStoreIndex

# Add the documents to the database and create Index / embeddings
index = VectorStoreIndex(nodes, storage_context=storage_context,show_progress =True)

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

...

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/1603 [00:00<?, ?it/s]

In [14]:
# Compress the vector store directory to a zip file to be able to download and use later.
!zip -r vectorstore-windowed.zip ai_tutor_knowledge

  adding: ai_tutor_knowledge/ (stored 0%)
  adding: ai_tutor_knowledge/8a0fd1e7-21aa-432e-853a-8cade26410a5/ (stored 0%)
  adding: ai_tutor_knowledge/8a0fd1e7-21aa-432e-853a-8cade26410a5/header.bin (deflated 55%)
  adding: ai_tutor_knowledge/8a0fd1e7-21aa-432e-853a-8cade26410a5/link_lists.bin (deflated 78%)
  adding: ai_tutor_knowledge/8a0fd1e7-21aa-432e-853a-8cade26410a5/data_level0.bin (deflated 17%)
  adding: ai_tutor_knowledge/8a0fd1e7-21aa-432e-853a-8cade26410a5/index_metadata.pickle (deflated 44%)
  adding: ai_tutor_knowledge/8a0fd1e7-21aa-432e-853a-8cade26410a5/length.bin (deflated 53%)
  adding: ai_tutor_knowledge/chroma.sqlite3 (deflated 90%)


# Load Indexes


**Note: If you didn’t create the vector store from scratch, please uncomment the three code blocks/cells below.**

In [4]:
# from huggingface_hub import hf_hub_download
# vectorstore = hf_hub_download(repo_id="jaiganesan/ai_tutor_knowledge", filename="vectorstore-windowed.zip",repo_type="dataset",local_dir="/content")

vectorstore-windowed.zip:   0%|          | 0.00/148M [00:00<?, ?B/s]

In [5]:
# !unzip vectorstore-windowed.zip

Archive:  vectorstore-windowed.zip
   creating: ai_tutor_knowledge/
   creating: ai_tutor_knowledge/0201aaa7-f5de-4efd-8594-fd5c929d6fdf/
  inflating: ai_tutor_knowledge/0201aaa7-f5de-4efd-8594-fd5c929d6fdf/header.bin  
  inflating: ai_tutor_knowledge/0201aaa7-f5de-4efd-8594-fd5c929d6fdf/data_level0.bin  
  inflating: ai_tutor_knowledge/0201aaa7-f5de-4efd-8594-fd5c929d6fdf/link_lists.bin  
  inflating: ai_tutor_knowledge/0201aaa7-f5de-4efd-8594-fd5c929d6fdf/length.bin  
  inflating: ai_tutor_knowledge/0201aaa7-f5de-4efd-8594-fd5c929d6fdf/index_metadata.pickle  
  inflating: ai_tutor_knowledge/chroma.sqlite3  


In [6]:
# import chromadb
# from llama_index.vector_stores.chroma import ChromaVectorStore
# from llama_index.core import VectorStoreIndex

# # Create your index
# db = chromadb.PersistentClient(path="./ai_tutor_knowledge")
# chroma_collection = db.get_or_create_collection("ai_tutor_knowledge")
# vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

# # Create your index
# index = VectorStoreIndex.from_vector_store(vector_store)

In [8]:
from llama_index.core.postprocessor import MetadataReplacementPostProcessor

query_engine = index.as_query_engine(
    llm=Settings.llm,
    # the target key defaults to `window` to match the node_parser's default
    node_postprocessors=[
        MetadataReplacementPostProcessor(target_metadata_key="window")
    ],
)

In [9]:
response = query_engine.query("Write about naive RAG and Speculative RAG?")
print(response)

Standard RAG methods and a more advanced approach called Self-Reflective RAG or Corrective RAG were compared to Speculative RAG.  Speculative RAG uses a smaller, specialized language model to create multiple drafts from different subsets of retrieved documents.  A larger, more general language model then verifies these drafts. This approach improves understanding of each document subset, reduces the effect of bias from long contexts, and speeds up the process by only having the larger model do a single verification step.  Speculative RAG consistently outperformed the other methods across multiple benchmarks, showing significant improvements in accuracy.



In [10]:
for idx, item in enumerate(response.source_nodes):
    print("Source ", idx + 1)
    print("Original Text:", item.node.metadata["original_text"])
    print("Window:", item.node.metadata["window"])
    print("----")

Source  1
Original Text: In this work, we introduce SPECULATIVE RAG – a framework that leverages a larger generalist LM to efficiently verify multiple RAG drafts produced in parallel by a smaller, distilled specialist LM. 
Window: **Abstract**Retrieval augmented generation (RAG) combines the generative abilities of large language models (LLMs) with external knowledge sources to provide more accurate and up-to-date responses.  Recent RAG advancements focus on improving retrieval outcomes through iterative LLM refinement or self-critique capabilities acquired through additional instruction tuning of LLMs.  In this work, we introduce SPECULATIVE RAG – a framework that leverages a larger generalist LM to efficiently verify multiple RAG drafts produced in parallel by a smaller, distilled specialist LM.  Each draft is generated from a distinct subset of retrieved documents, offering diverse perspectives on the evidence while reducing input token counts per draft.  This approach enhances comp